# Trajectory Viewer
Interactive tool for browsing saved Crescendomation trajectories.

In [1]:
import json
from pathlib import Path
import torch as t
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

OUTPUT_DIR = Path("results/crescendo_trajectories")

# Find all saved trajectory files
traj_files = sorted(OUTPUT_DIR.glob("trajectories_*.json"))
if not traj_files:
    print("No trajectory files found in", OUTPUT_DIR)
else:
    print(f"Found {len(traj_files)} trajectory file(s):")
    for f in traj_files:
        print(f"  {f.name}")

Found 2 trajectory file(s):
  trajectories_20260220_050157.json
  trajectories_20260221_190645.json


In [ ]:
# Load selected file
file_dropdown = widgets.Dropdown(
    options=[(f.name, f) for f in traj_files],
    description="File:",
    style={"description_width": "50px"},
    layout=widgets.Layout(width="500px"),
)

traj_dropdown = widgets.Dropdown(description="Trajectory:", style={"description_width": "80px"}, layout=widgets.Layout(width="700px"))
turn_dropdown = widgets.Dropdown(description="Turn:", style={"description_width": "50px"}, layout=widgets.Layout(width="500px"))
output = widgets.Output()

# State
trajectories = []


def load_file(change=None):
    global trajectories
    path = file_dropdown.value
    with open(path, "r", encoding="utf-8") as f:
        trajectories = json.load(f)
    traj_dropdown.options = [
        (f"[{i}] {tr['label'].upper():10s} | score={tr['max_score_normalized']:.0f} | {tr['goal'][:60]}...", i)
        for i, tr in enumerate(trajectories)
    ]


def on_traj_change(change=None):
    idx = traj_dropdown.value
    if idx is None:
        return
    traj = trajectories[idx]
    turn_dropdown.options = [
        (f"Turn {t['turn_idx']}  score={t['normalized_score']:.0f}{'  [BACKTRACK]' if t['was_backtracked'] else ''}", i)
        for i, t in enumerate(traj["turns"])
    ]
    show_turn()


def show_turn(change=None):
    traj_idx = traj_dropdown.value
    turn_idx = turn_dropdown.value
    if traj_idx is None or turn_idx is None:
        return
    traj = trajectories[traj_idx]
    turn = traj["turns"][turn_idx]

    with output:
        clear_output(wait=True)

        # Trajectory header
        label_color = "red" if traj["label"] == "jailbroken" else "green"
        scores = [t["normalized_score"] for t in traj["turns"] if not t["was_backtracked"]]
        score_str = " -> ".join(str(int(s)) for s in scores)

        html = f"""
        <div style='font-family: monospace; padding: 10px; background: #1e1e1e; color: #d4d4d4; border-radius: 8px;'>
            <h3 style='color: #569cd6; margin-top:0;'>Trajectory {traj_idx}</h3>
            <b>Goal:</b> {traj['goal']}<br>
            <b>Label:</b> <span style='color:{label_color}; font-weight:bold;'>{traj['label'].upper()}</span>
            &nbsp;|&nbsp; <b>Max Score:</b> {traj['max_score_normalized']:.0f}/10
            &nbsp;|&nbsp; <b>Turns:</b> {traj['num_turns']}
            &nbsp;|&nbsp; <b>Backtracks:</b> {traj['num_backtracks']}<br>
            <b>Score trajectory:</b> {score_str}
            <hr style='border-color:#444;'>

            <h3 style='color: #569cd6;'>Turn {turn['turn_idx']}
                {'<span style="color:orange;"> [BACKTRACKED]</span>' if turn['was_backtracked'] else ''}
            </h3>
            <b>Score:</b> {turn['normalized_score']:.0f}/10 (raw: {turn['raw_score']})<br><br>

            <b style='color:#dcdcaa;'>User Prompt:</b>
            <div style='background:#2d2d2d; padding:8px; margin:4px 0 12px 0; border-left:3px solid #569cd6; white-space:pre-wrap;'>{turn['user_prompt']}</div>

            <b style='color:#dcdcaa;'>Model Response:</b>
            <div style='background:#2d2d2d; padding:8px; margin:4px 0 12px 0; border-left:3px solid #4ec9b0; white-space:pre-wrap;'>{turn['model_response']}</div>
        </div>
        """
        display(HTML(html))


# Wire up callbacks
file_dropdown.observe(lambda c: load_file() if c["name"] == "value" else None)
traj_dropdown.observe(lambda c: on_traj_change() if c["name"] == "value" else None)
turn_dropdown.observe(lambda c: show_turn() if c["name"] == "value" else None)

# Initial load
load_file()

display(widgets.VBox([
    file_dropdown,
    traj_dropdown,
    turn_dropdown,
    output,
]))

In [3]:
# Show all turns for current trajectory at once
btn = widgets.Button(description="Show All Turns", button_style="info")
all_output = widgets.Output()

def show_all(b):
    traj_idx = traj_dropdown.value
    if traj_idx is None:
        return
    traj = trajectories[traj_idx]
    with all_output:
        clear_output(wait=True)
        for turn in traj["turns"]:
            bt = " [BACKTRACKED]" if turn["was_backtracked"] else ""
            score_color = "red" if turn["normalized_score"] >= 8 else ("orange" if turn["normalized_score"] >= 5 else "green")
            html = f"""
            <div style='font-family:monospace; padding:6px; margin:4px 0; background:#1e1e1e; color:#d4d4d4; border-radius:4px; border-left:4px solid {score_color};'>
                <b>Turn {turn['turn_idx']}</b> — Score: <span style='color:{score_color};'>{turn['normalized_score']:.0f}</span>{bt}<br>
                <b>User:</b> {turn['user_prompt'][:200]}{'...' if len(turn['user_prompt']) > 200 else ''}<br>
                <b>Model:</b> {turn['model_response'][:200]}{'...' if len(turn['model_response']) > 200 else ''}
            </div>
            """
            display(HTML(html))

btn.on_click(show_all)
display(btn, all_output)

Button(button_style='info', description='Show All Turns', style=ButtonStyle())

Output()